In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create the large multiclass network
model = Network('MyNetwork')

In [3]:
# Block 1: Network nodes
node = np.empty(6, dtype=object)
node[0] = Source(model, 'Source')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)  # WebServer
node[2] = Queue(model, 'Queue2', SchedStrategy.FCFS)  # Storage1
node[3] = Queue(model, 'Queue3', SchedStrategy.PS)    # Storage2
node[4] = Queue(model, 'Queue4', SchedStrategy.FCFS)  # Storage3
node[5] = Sink(model, 'Sink')

In [4]:
# Block 2: Job classes with different characteristics
jobclass = np.empty(3, dtype=object)
jobclass[0] = OpenClass(model, 'Class1', 0)  # High-priority requests
jobclass[1] = OpenClass(model, 'Class2', 0)  # Medium-priority requests
jobclass[2] = OpenClass(model, 'Class3', 0)  # Low-priority requests

In [5]:
# Arrival processes - different arrival rates per class
node[0].set_arrival(jobclass[0], Exp.fitMean(5.000000))  # Class1: mean inter-arrival = 5.0 (rate = 0.2)
node[0].set_arrival(jobclass[1], Exp.fitMean(8.000000))  # Class2: mean inter-arrival = 8.0 (rate = 0.125)
node[0].set_arrival(jobclass[2], Exp.fitMean(7.000000))  # Class3: mean inter-arrival = 7.0 (rate = 0.143)

In [6]:
# Service processes at WebServer (Queue1) - front-end processing
node[1].set_service(jobclass[0], Exp.fitMean(0.300000))  # Class1: fast processing
node[1].set_service(jobclass[1], Exp.fitMean(0.500000))  # Class2: medium processing
node[1].set_service(jobclass[2], Exp.fitMean(0.600000))  # Class3: slower processing

In [7]:
# Service processes at Storage1 (Queue2) - database access
node[2].set_service(jobclass[0], Exp.fitMean(1.100000))  # Class1: complex queries
node[2].set_service(jobclass[1], Exp.fitMean(1.300000))  # Class2: medium queries
node[2].set_service(jobclass[2], Exp.fitMean(1.500000))  # Class3: simple queries

In [8]:
# Service processes at Storage2 (Queue3) - file system access with PS
node[3].set_service(jobclass[0], Exp.fitMean(2.000000))  # Class1: large files
node[3].set_service(jobclass[1], Exp.fitMean(2.100000))  # Class2: medium files
node[3].set_service(jobclass[2], Exp.fitMean(1.900000))  # Class3: small files

In [9]:
# Service processes at Storage3 (Queue4) - cache access
node[4].set_service(jobclass[0], Exp.fitMean(1.500000))  # Class1: cache miss handling
node[4].set_service(jobclass[1], Exp.fitMean(0.900000))  # Class2: cache hits
node[4].set_service(jobclass[2], Exp.fitMean(2.300000))  # Class3: cache updates

In [10]:
# Block 3: Complex routing topology with feedback loops
P = model.init_routing_matrix()

# Class1 routing
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)        # Source → WebServer
P.set(jobclass[0], jobclass[0], node[1], node[2], 0.25)       # WebServer → Storage1 (25%)
P.set(jobclass[0], jobclass[0], node[1], node[3], 0.25)       # WebServer → Storage2 (25%)
P.set(jobclass[0], jobclass[0], node[1], node[4], 0.25)       # WebServer → Storage3 (25%)
P.set(jobclass[0], jobclass[0], node[1], node[5], 0.25)       # WebServer → Sink (25%)
P.set(jobclass[0], jobclass[0], node[2], node[1], 1.0)        # Storage1 → WebServer (feedback)
P.set(jobclass[0], jobclass[0], node[3], node[1], 1.0)        # Storage2 → WebServer (feedback)
P.set(jobclass[0], jobclass[0], node[4], node[1], 1.0)        # Storage3 → WebServer (feedback)

# Class2 routing
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)        # Source → WebServer
P.set(jobclass[1], jobclass[1], node[1], node[2], 0.25)       # WebServer → Storage1 (25%)
P.set(jobclass[1], jobclass[1], node[1], node[3], 0.25)       # WebServer → Storage2 (25%)
P.set(jobclass[1], jobclass[1], node[1], node[4], 0.25)       # WebServer → Storage3 (25%)
P.set(jobclass[1], jobclass[1], node[1], node[5], 0.25)       # WebServer → Sink (25%)
P.set(jobclass[1], jobclass[1], node[2], node[1], 1.0)        # Storage1 → WebServer (feedback)
P.set(jobclass[1], jobclass[1], node[3], node[1], 1.0)        # Storage2 → WebServer (feedback)
P.set(jobclass[1], jobclass[1], node[4], node[1], 1.0)        # Storage3 → WebServer (feedback)

# Class3 routing
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)        # Source → WebServer
P.set(jobclass[2], jobclass[2], node[1], node[2], 0.25)       # WebServer → Storage1 (25%)
P.set(jobclass[2], jobclass[2], node[1], node[3], 0.25)       # WebServer → Storage2 (25%)
P.set(jobclass[2], jobclass[2], node[1], node[4], 0.25)       # WebServer → Storage3 (25%)
P.set(jobclass[2], jobclass[2], node[1], node[5], 0.25)       # WebServer → Sink (25%)
P.set(jobclass[2], jobclass[2], node[2], node[1], 1.0)        # Storage1 → WebServer (feedback)
P.set(jobclass[2], jobclass[2], node[3], node[1], 1.0)        # Storage2 → WebServer (feedback)
P.set(jobclass[2], jobclass[2], node[4], node[1], 1.0)        # Storage3 → WebServer (feedback)

model.link(P)

In [11]:
# Configure solver options for the large network
options = lineDefaults()  # Call the function to get SolverOptions object
options['seed'] = 23000
options['samples'] = int(1e6)  # High sample count for accuracy
options['cutoff'] = 1         # Population cutoff for finite state approximation

In [ ]:
# Aligned with JAR test scenarios for oqn_fourqueues
# JAR tests: CTMC(seed=23000, cutoff=1), MVA(seed=23000), MAM(seed=23000), JMT(seed=23000, samples=1000000)

solver = np.array([], dtype=object)

# CTMC with seed=23000, cutoff=1 (matches JAR)
solver = np.append(solver, SolverCTMC(model, seed=23000, cutoff=1))

# MVA with seed=23000 (matches JAR)
solver = np.append(solver, SolverMVA(model, seed=23000))

# MAM with seed=23000 (matches JAR)
solver = np.append(solver, SolverMAM(model, seed=23000))

# JMT with seed=23000, samples=1000000 (matches JAR)
solver = np.append(solver, SolverJMT(model, seed=23000, samples=1000000))

In [13]:
# Execute all solvers and collect results
AvgTable = np.empty(len(solver), dtype=object)

for s in range(len(solver)):
    AvgTable[s] = solver[s].get_avg_table()

CTMC analysis [method: default, lang: java, env: 21.0.8] completed in 0.309663s.
   Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Source   Class1  0.0000  0.0000  0.0000  0.0000  0.0000  0.0782
1   Source   Class2  0.0000  0.0000  0.0000  0.0000  0.0000  0.0611
2   Source   Class3  0.0000  0.0000  0.0000  0.0000  0.0000  0.0599
3   Queue1   Class1  0.1462  0.0938  0.4677  1.8707  0.3127  0.3127
4   Queue1   Class2  0.1568  0.1223  0.6413  2.5653  0.2445  0.2445
5   Queue1   Class3  0.1700  0.1436  0.7100  2.8402  0.2394  0.2394
6   Queue2   Class1  0.1090  0.0860  1.3940  1.3940  0.0782  0.0782
7   Queue2   Class2  0.0960  0.0795  1.5698  1.5698  0.0611  0.0611
8   Queue2   Class3  0.1030  0.0898  1.7216  1.7216  0.0599  0.0599
9   Queue3   Class1  0.2029  0.1563  2.5963  2.5963  0.0782  0.0782
10  Queue3   Class2  0.1713  0.1284  2.8018  2.8018  0.0611  0.0611
11  Queue3   Class3  0.1541  0.1137  2.5753  2.5753  0.0599  0.0599
12  Queue4   Class1  0.1510  0.1172